# Lab 9b: Local Tracing with AI Toolkit

This notebook demonstrates **local tracing** using AI Toolkit's OTLP collector in VS Code.

## When to Use This Notebook vs lab9_tracing_observability.ipynb

| This Notebook (AI Toolkit) | Main Lab 9 (Azure Monitor) |
|---------------------------|---------------------------|
| Local development only | Production-ready |
| Traces in VS Code | Traces in Foundry portal |
| No cloud costs | Requires Azure subscription |
| Session-only (not persisted) | 90-day retention |
| Instant visibility | 1-2 min latency |

## How It Works

```
Your Code → OpenTelemetry SDK → OTLP Exporter → http://localhost:4318 → AI Toolkit
```

AI Toolkit runs a local **OTLP collector** that receives traces via HTTP on port 4318.

## Prerequisites

1. **AI Toolkit extension** installed in VS Code
2. Open **AI Toolkit** in VS Code sidebar
3. Click **Tracing** in the tree view
4. Click **Start Collector** (must show "Status: Running")
5. Run all cells below **in order**
6. Click **Refresh** in Tracing view to see traces

> ⚠️ **Important**: This notebook uses a different `TracerProvider` than lab9. Do not mix them in the same kernel session.

In [1]:
# Cell 1: Setup environment
import os
from pathlib import Path
from dotenv import load_dotenv

# Ensure Azure CLI is in PATH
for p in ["/opt/homebrew/bin", "/usr/local/bin"]:
    if p not in os.environ.get("PATH", ""):
        os.environ["PATH"] = p + ":" + os.environ.get("PATH", "")

# Load .env
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded {env_path.resolve()}")

endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"✅ Endpoint: {endpoint[:50]}..." if endpoint else "❌ Missing AZURE_AI_PROJECT_ENDPOINT")
print(f"✅ Model: {model}")

✅ Loaded /Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/.env
✅ Endpoint: https://nw-fy-26.services.ai.azure.com/api/project...
✅ Model: gpt-4.1


In [2]:
# Cell 2: Configure AI Toolkit Tracing (OTLP to localhost:4318)
import os

# Enable content capture
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"

from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

# Configure OTLP exporter to AI Toolkit local collector
resource = Resource.create({"service.name": "lab9-ai-toolkit"})
tracer_provider = TracerProvider(resource=resource)

# AI Toolkit HTTP endpoint
otlp_exporter = OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
tracer_provider.add_span_processor(BatchSpanProcessor(otlp_exporter))

# Set as global tracer provider
trace.set_tracer_provider(tracer_provider)

# Instrument OpenAI SDK
OpenAIInstrumentor().instrument()

print("✅ AI Toolkit Tracing Configured!")
print("   Endpoint: http://localhost:4318/v1/traces")
print("   Service: lab9-ai-toolkit")
print()
print("📋 Make sure AI Toolkit collector is running in VS Code!")

✅ AI Toolkit Tracing Configured!
   Endpoint: http://localhost:4318/v1/traces
   Service: lab9-ai-toolkit

📋 Make sure AI Toolkit collector is running in VS Code!


In [3]:
# Cell 3: Create OpenAI client
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Get AI Services base URL
ai_services_base = endpoint.split("/api/projects")[0]

# Create token provider
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

# Create client
openai_client = AzureOpenAI(
    azure_endpoint=ai_services_base,
    api_version="2024-10-21",
    azure_ad_token_provider=token_provider
)

print(f"✅ OpenAI client created")
print(f"   Endpoint: {ai_services_base}")
print(f"   Model: {model}")

✅ OpenAI client created
   Endpoint: https://nw-fy-26.services.ai.azure.com
   Model: gpt-4.1


In [4]:
# Cell 4: Simple traced request
tracer = trace.get_tracer(__name__)

print("🔍 Making a traced request...")
print()

with tracer.start_as_current_span("simple-chat-request") as span:
    span.set_attribute("user.query", "Say hello")
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Be brief."},
            {"role": "user", "content": "Say hello in 3 different languages."}
        ],
        temperature=0.7,
        max_tokens=100
    )
    
    answer = response.choices[0].message.content
    span.set_attribute("response.preview", answer[:50])
    print(f"📝 Response:\n{answer}")

print()
print("✅ Trace sent to AI Toolkit!")
print("   Click 'Refresh' in AI Toolkit Tracing view to see it")

🔍 Making a traced request...

📝 Response:
Hello!  
Hola!  
Bonjour!

✅ Trace sent to AI Toolkit!
   Click 'Refresh' in AI Toolkit Tracing view to see it


In [5]:
# Cell 5: Complete agent session with tool calls
import json

print("🤖 Running Complete Agent Session")
print("=" * 50)
print()

# Tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather for a city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculate math expression",
            "parameters": {
                "type": "object",
                "properties": {"expression": {"type": "string"}},
                "required": ["expression"]
            }
        }
    }
]

def execute_tool(name, args):
    with tracer.start_as_current_span(f"tool_{name}") as s:
        s.set_attribute("tool.name", name)
        s.set_attribute("tool.args", json.dumps(args))
        if name == "get_weather":
            result = f"Weather in {args.get('city', '?')}: 18°C, Sunny"
        elif name == "calculate":
            try:
                result = str(eval(args.get('expression', '0')))
            except:
                result = "Error"
        else:
            result = "Unknown"
        s.set_attribute("tool.result", result)
        return result

# Agent loop
query = "What's the weather in Paris? And what's 25 * 4?"
print(f"📝 Query: {query}")
print()

with tracer.start_as_current_span("agent_session") as session:
    session.set_attribute("user.query", query)
    messages = [
        {"role": "system", "content": "Use tools to answer questions."},
        {"role": "user", "content": query}
    ]
    
    # Step 1: Planning
    with tracer.start_as_current_span("planning"):
        resp = openai_client.chat.completions.create(
            model=model, messages=messages, tools=tools, tool_choice="auto"
        )
        msg = resp.choices[0].message
        tool_calls = msg.tool_calls or []
        print(f"🔧 Tools requested: {len(tool_calls)}")
    
    # Step 2: Execute tools
    if tool_calls:
        messages.append(msg)
        with tracer.start_as_current_span("tool_execution"):
            for tc in tool_calls:
                name = tc.function.name
                args = json.loads(tc.function.arguments)
                print(f"   → {name}({args})")
                result = execute_tool(name, args)
                print(f"     = {result}")
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
    
    # Step 3: Final response
    with tracer.start_as_current_span("response_generation"):
        final = openai_client.chat.completions.create(model=model, messages=messages)
        answer = final.choices[0].message.content

print()
print(f"💬 Agent Response:\n{answer}")
print()
print("=" * 50)
print("✅ Full trace captured!")
print("   Click 'Refresh' in AI Toolkit → Tracing")
print()
print("📊 You should see:")
print("   agent_session")
print("   ├── planning")
print("   ├── tool_execution")
print("   │   ├── tool_get_weather")
print("   │   └── tool_calculate")  
print("   └── response_generation")

🤖 Running Complete Agent Session

📝 Query: What's the weather in Paris? And what's 25 * 4?

🔧 Tools requested: 2
   → get_weather({'city': 'Paris'})
     = Weather in Paris: 18°C, Sunny
   → calculate({'expression': '25 * 4'})
     = 100

💬 Agent Response:
The weather in Paris is 18°C and sunny.

25 * 4 equals 100.

✅ Full trace captured!
   Click 'Refresh' in AI Toolkit → Tracing

📊 You should see:
   agent_session
   ├── planning
   ├── tool_execution
   │   ├── tool_get_weather
   │   └── tool_calculate
   └── response_generation


In [6]:
# Cell 6: Flush traces (force send)
import time

print("⏳ Flushing traces...")
tracer_provider.force_flush()
time.sleep(1)

print("✅ Traces flushed!")
print()
print("👉 Now click 'Refresh' in AI Toolkit Tracing view")

⏳ Flushing traces...
✅ Traces flushed!

👉 Now click 'Refresh' in AI Toolkit Tracing view
